In [1]:
!pip install neuralprophet

In [2]:
import numpy as np 
import pandas as pd
import os
from neuralprophet import NeuralProphet
#%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
from scipy import stats
import seaborn as sns
import math
#import xgboost
#from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [3]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [4]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
#import os
#iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

#if iskaggle:
#    !pip install -Uqq fastai duckduckgo_search

In [5]:
#from fastdownload import download_url
#dest = 'cat.jpg'
#download_url('https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46744842/1/?bust=1575403984&width=300', dest, show_progress=False)

In [6]:
#from fastai.vision.all import *
#im = Image.open(dest)#"/kaggle/input/cat-breeds-dataset/images/Balinese/10895463_920.jpg")
#im.to_thumb(256,256)

In [7]:
#path = Path('/kaggle/input/cat-gender/meow')
#path = Path('/kaggle/input/lviv-temperature/')

In [ ]:
#dls = DataBlock(
#    blocks=(ImageBlock, CategoryBlock), 
#    get_items=get_image_files, 
#    splitter=RandomSplitter(valid_pct=0.2, seed=42),
#    get_y=parent_label,
#    item_tfms=[Resize(192, method='squish')]
#).dataloaders(path, bs=32)

#dls.show_batch(max_n=6)

In [ ]:
#learn = vision_learner(dls, resnet18, metrics=error_rate)
#learn.fine_tune(3)

In [8]:
data = pd.read_csv('/kaggle/input/lviv-temperature/lviv_temp.csv', header=0,names=["date","year","month","day","dayofyear","data"])
data["date"]=pd.to_datetime(data["date"])
data.head()


In [9]:
data.dtypes

In [10]:
g = data[["data","date"]]
g.index=g["date"]
g["data"].plot(style='.',
        figsize=(15, 5),
        color=color_pal[0],
        title='Average temperature in lviv C')
plt.show()

In [11]:
train = g.loc[g.index < '01-01-2010']
test = g.loc[g.index >= '01-01-2010']

fig, ax = plt.subplots(figsize=(15, 5))
train["data"].plot(ax=ax, label='Training Set', title='Data Train/Test Split')
test["data"].plot(ax=ax, label='Test Set')
ax.axvline('01-01-2010', color='black', ls='--')
ax.legend(['Training Set', 'Test Set'])
plt.show()

In [12]:
plt.plot(data["date"], data["data"])
plt.show()

In [13]:
new_column = data[["date","data"]][:-3000]
new_column.columns=["ds","y"]
new_column

In [14]:
n = NeuralProphet()
model = n.fit(new_column, freq='D')#, epochs=5000)

In [15]:
future = n.make_future_dataframe(new_column, periods=10000)
forecast = n.predict(future)
forecast.tail()
future
forecast

In [16]:
plot = n.plot(forecast)

In [17]:
pred = data[["date","data"]][-3000:]
pred.columns=['ds','y']
predictions = n.predict(pred)
predictions.index=pred.index
#df = data.merge(test[['prediction']], how='left', left_index=True, right_index=True)
ax = data[['data']].plot(figsize=(15, 5))
#plt.plot(data["date"], data["data"])
predictions["y"].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Dat and Prediction')
plt.show()
#predictions

In [22]:
k = pd.DataFrame({"ds": [pd.to_datetime("07-06-2200"),pd.to_datetime("07-07-2200")], "y": [None, None]})
dyp = data[["date","data"]][-2:]
dyp.columns=['ds','y']
fore = n.predict(k)
fore

In [26]:
n.path = "/kaggle/input"

In [27]:
n.export()

In [32]:
import pickle
pkl_path = "/kaggle/working/fore.pkl"
with open(pkl_path, "wb") as f:
    # Pickle the 'Prophet' model using the highest protocol available.
    pickle.dump(n, f)

# save the dataframe
forecast.to_pickle("/kaggle/working/forecast.pkl")
print("*** Data Saved ***")

In [35]:
f = pd.read_pickle("/kaggle/working/fore.pkl")
f.predict(k)

In [36]:
l = load_learner("/kaggle/working/fore.pkl")